## Miami

In [53]:
df_2013_to_2017_miami=df_2013_to_2017[
    df_2013_to_2017["zip_code"].isin(miami_zip)]

df_2013_to_2017_miami=df_2013_to_2017_miami.drop(["zip_code","year"],axis=1)
#dropping unique identifiers

In [54]:
df_2018_miami=df_2018[df_2018["zip_code"].isin(miami_zip)]
df_2018_miami=df_2018_miami.drop(["zip_code","year"],axis=1)

### Linear Regression

In [55]:
ols_miami = linear_model.LinearRegression()
ols_miami.fit(df_2013_to_2017_miami.drop(columns = 'Monthly_Avg_Zip'), 
              df_2013_to_2017_miami.Monthly_Avg_Zip)

LinearRegression()

In [56]:
ols_miami.score(df_2018_miami.drop(columns = 'Monthly_Avg_Zip'), 
               df_2018_miami.Monthly_Avg_Zip)

0.7440131637259504

In [57]:
rmse_test_ols_miami=np.sqrt(
    mean_squared_error
    (df_2018_miami.Monthly_Avg_Zip, 
    ols_miami.predict(df_2018_miami.drop(columns = 'Monthly_Avg_Zip'))
    )
)
print(rmse_test_ols_miami)

122.1203309008359


### Lasso Regression

In [58]:
lasso_miami=Lasso()

In [59]:
lasso_miami.fit(df_2013_to_2017_miami.drop(columns = 'Monthly_Avg_Zip'), 
              df_2013_to_2017_miami.Monthly_Avg_Zip)

Lasso()

In [60]:
rmse_test_lasso_miami=np.sqrt(
    mean_squared_error
    (df_2018_miami.Monthly_Avg_Zip, 
    lasso_miami.predict(df_2018_miami.drop(columns = 'Monthly_Avg_Zip'))
    )
)
print(rmse_test_lasso_miami)

140.25559124705813


In [61]:
from sklearn.model_selection import GridSearchCV

lasso_params = {
    "alpha": list(np.logspace(-8,2,11))
    ,"max_iter": [10, 100, 1000, 10000]
    , "tol": list(np.logspace(-8,0,9))
    
}

lasso_miami_gs = GridSearchCV(lasso_miami, lasso_params, cv=kfold)
lasso_miami_gs.fit(df_2013_to_2017_miami.drop(columns = 'Monthly_Avg_Zip'), 
              df_2013_to_2017_miami.Monthly_Avg_Zip)
print("The best parameters are: ", lasso_miami_gs.best_params_)

The best parameters are:  {'alpha': 0.1, 'max_iter': 1000, 'tol': 0.01}


In [62]:
print("The best parameters are: ", lasso_miami_gs.best_params_)

The best parameters are:  {'alpha': 0.1, 'max_iter': 1000, 'tol': 0.01}


In [63]:
lasso_miami_gs.score(df_2018_miami.drop(columns = 'Monthly_Avg_Zip'), 
               df_2018_miami.Monthly_Avg_Zip)

0.7448921061493472

In [64]:
lasso_miami_tuned=Lasso(**lasso_miami_gs.best_params_)

In [65]:
lasso_miami_tuned.fit(df_2013_to_2017_miami.drop(columns = 'Monthly_Avg_Zip'), 
              df_2013_to_2017_miami.Monthly_Avg_Zip)

Lasso(alpha=0.1, tol=0.01)

In [66]:
rmse_test_lasso_miami=np.sqrt(
    mean_squared_error
    (df_2018_miami.Monthly_Avg_Zip, 
    lasso_miami_tuned.predict(df_2018_miami.drop(columns = 'Monthly_Avg_Zip'))
    )
)
print(rmse_test_lasso_miami)

121.91049777804491


In [67]:
df_2018_miami.Monthly_Avg_Zip.describe()

count      66.000000
mean     1705.161780
std       243.217290
min      1316.166667
25%      1513.875000
50%      1677.041667
75%      1815.833333
max      2294.250000
Name: Monthly_Avg_Zip, dtype: float64

### Cross Validation on the train set (data in 2013-2017)

In [68]:
X_train, X_test, y_train, y_test = train_test_split(
    df_2013_to_2017_miami.drop(columns = 'Monthly_Avg_Zip'), 
    df_2013_to_2017_miami.Monthly_Avg_Zip, 
    test_size=0.3, random_state=0)

In [69]:
lasso_miami_tuned_scores = cross_val_score(lasso_miami_tuned, 
                                           X_train, y_train, cv=kfold)
print(lasso_miami_tuned_scores)
np.mean(lasso_miami_tuned_scores)

[0.82115597 0.637582   0.78441048]


0.7477161499442477

In [70]:
lasso_miami_tuned.score(X_test, y_test)

0.9038964602349625

### Random Forest

In [71]:
rf_miami = RandomForestRegressor(random_state=0)
rf_miami.fit(df_2013_to_2017_miami.drop(columns = 'Monthly_Avg_Zip'), 
             df_2013_to_2017_miami.Monthly_Avg_Zip)

RandomForestRegressor(random_state=0)

In [72]:
feature_importances_miami = pd.Series(rf_miami.feature_importances_, 
                                index=df_2013_to_2017_miami.drop(columns = 'Monthly_Avg_Zip').columns)
feature_importances_miami=feature_importances_miami.sort_values(ascending=False)
feature_importances_miami.head(10)

owner_occupied_housing_units_median_value    0.454522
income_200000_or_more                        0.043426
commute_less_10_mins                         0.032211
male_male_households                         0.028593
not_us_citizen_pop                           0.027744
gini_index                                   0.025381
dwellings_50_or_more_units                   0.024946
dwellings_5_to_9_units                       0.023235
children_in_single_female_hh                 0.022425
male_35_to_39                                0.019737
dtype: float64